In [1]:
import os

# set current dir to Mask3D dir
mask3d_dir = "/workspace/Mask3D_adapted"
os.chdir(mask3d_dir)

import logging
import os
from hashlib import md5
from uuid import uuid4
import hydra
from dotenv import load_dotenv
from omegaconf import DictConfig, OmegaConf
from trainer.trainer import InstanceSegmentation, RegularCheckpointing
from pytorch_lightning.callbacks import ModelCheckpoint
from utils.utils import (
    flatten_dict,
    load_baseline_model,
    load_checkpoint_with_missing_or_exsessive_keys,
    load_backbone_checkpoint_with_missing_or_exsessive_keys,
)
from datasets.utils import *
from pytorch_lightning import Trainer, seed_everything
import numpy as np

import logging
from itertools import product
from pathlib import Path
from random import random, sample, uniform
from typing import List, Optional, Tuple, Union
from random import choice
from copy import deepcopy
from random import randrange


import numpy
import torch
from datasets.random_cuboid import RandomCuboid

import albumentations as A
import numpy as np
import scipy
import volumentations as V
import yaml

# from yaml import CLoader as Loader
from torch.utils.data import Dataset
from datasets.scannet200.scannet200_constants import (
    SCANNET_COLOR_MAP_200,
    SCANNET_COLOR_MAP_20,
)
from datasets.scannetpp.scannetpp_constants import (SCANNETPP_COLOR_MAP)

# models 
from models.mask3d import *

def elastic_distortion(pointcloud, granularity, magnitude):
    """Apply elastic distortion on sparse coordinate space.

    pointcloud: numpy array of (number of points, at least 3 spatial dims)
    granularity: size of the noise grid (in same scale[m/cm] as the voxel grid)
    magnitude: noise multiplier
    """
    blurx = np.ones((3, 1, 1, 1)).astype("float32") / 3
    blury = np.ones((1, 3, 1, 1)).astype("float32") / 3
    blurz = np.ones((1, 1, 3, 1)).astype("float32") / 3
    coords = pointcloud[:, :3]
    coords_min = coords.min(0)

    # Create Gaussian noise tensor of the size given by granularity.
    noise_dim = ((coords - coords_min).max(0) // granularity).astype(int) + 3
    noise = np.random.randn(*noise_dim, 3).astype(np.float32)

    # Smoothing.
    for _ in range(2):
        noise = scipy.ndimage.filters.convolve(
            noise, blurx, mode="constant", cval=0
        )
        noise = scipy.ndimage.filters.convolve(
            noise, blury, mode="constant", cval=0
        )
        noise = scipy.ndimage.filters.convolve(
            noise, blurz, mode="constant", cval=0
        )

    # Trilinear interpolate noise filters for each spatial dimensions.
    ax = [
        np.linspace(d_min, d_max, d)
        for d_min, d_max, d in zip(
            coords_min - granularity,
            coords_min + granularity * (noise_dim - 2),
            noise_dim,
        )
    ]
    interp = scipy.interpolate.RegularGridInterpolator(
        ax, noise, bounds_error=0, fill_value=0
    )
    pointcloud[:, :3] = coords + interp(coords) * magnitude
    return pointcloud

/opt/conda/envs/mask3d_cuda113/lib/python3.10/site-packages/MinkowskiEngine-0.5.4-py3.10-linux-x86_64.egg/MinkowskiEngine/__init__.py:36: UserWarning: The environment variable `OMP_NUM_THREADS` not set. MinkowskiEngine will automatically set `OMP_NUM_THREADS=16`. If you want to set `OMP_NUM_THREADS` manually, please export it on the command line before running a python script. e.g. `export OMP_NUM_THREADS=12; python your_program.py`. It is recommended to set it below 24.
  warnings.warn(


In [2]:
import MinkowskiEngine as ME
import numpy as np
import torch
from random import random
from datasets.semseg import SemanticSegmentationDataset

In [3]:
dataaset = SemanticSegmentationDataset(
        dataset_name="scannetpp",
        data_dir = "data/processed/scannetpp",
        label_db_filepath = "data/processed/scannetpp/label_database.yaml",
        color_mean_std = "data/processed/scannetpp/color_mean_std.yaml",
        mode = "train_validation",
        add_colors = True,
        add_normals = False,
        add_raw_coordinates = True,
        add_instance = True,
        num_labels = 100,
        data_percent = 1.0,
        ignore_label = 255,
        volume_augmentations_path = None,
        image_augmentations_path = None,
        instance_oversampling=0,
        place_around_existing=False,
        max_cut_region=0,
        point_per_cut=0,
        flip_in_center=False,
        noise_rate=0.0,
        resample_points=0.0,
        cache_data=False,
        add_unlabeled_pc=False,
        task="instance_segmentation",
        cropping=False,
        cropping_args=None,
        is_tta=False,
        crop_min_size=20000,
        crop_length=6.0,
        cropping_v1=True,
        reps_per_epoch=1,
        area=-1,
        on_crops=False,
        eval_inner_core=-1,
        filter_out_classes=[0, 1 , 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 255],
        label_offset=16,
        add_clip=False,
        is_elastic_distortion=True,
        color_drop=0.0,
)
collater = VoxelizeCollate(
        ignore_label=255,
        voxel_size=0.02,
        mode="train_mode",
        small_crops=False,
        very_small_crops=False,
        batch_instance=False,
        probing=False,
        task="instance_segmentation",
        ignore_class_threshold=100,
        filter_out_classes=[0, 1 , 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 255],
        label_offset=16,
        num_queries=150,
)

/workspace/Mask3D_adapted/datasets/semseg.py:711: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  file = yaml.load(f)


In [9]:
labels = dataaset._load_yaml("/workspace/Mask3D_adapted/data/processed/scannetpp/label_database.yaml")
_labels = dataaset._select_correct_labels(labels, 100)

In [11]:
num_labels = 100
number_of_validation_labels = 0
number_of_all_labels = 0
for (
    k,
    v,
) in labels.items():
    number_of_all_labels += 1
    if v["validation"]:
        number_of_validation_labels += 1

if num_labels == number_of_all_labels:
    pass
elif num_labels == number_of_validation_labels:
    valid_labels = dict()
    for (
        k,
        v,
    ) in labels.items():
        if v["validation"]:
            valid_labels.update({k: v})

In [16]:
number_of_validation_labels

84

In [13]:
number_of_validation_labels

84

In [7]:
target_scenes = ["0d2ee665be", "3f15a9266d", "7b6477cb95", "3e8bba0176", "5f99900f09", "3db0a1c8f3", "5ee7c22ba0", "1ada7a0617", "5eb31827b7", "7bc286c1b6"]

In [ ]:
for i in range(len(dataaset)):
    if 

In [19]:
np.unique(dataaset[9][0])

Scene:  7bc286c1b6


array([-8.9327223e-06, -1.7310082e-07,  1.4993851e-05, ...,
        2.8688519e+00,  2.8688748e+00,  2.8690846e+00], dtype=float32)

In [20]:
np.unique(dataaset[9][2][:, 0])

Scene:  7bc286c1b6


array([ 7, 17, 19, 43, 47, 53, 59, 60, 83, 95], dtype=int32)

In [21]:
np.unique(dataaset[9][2][:, 1])

Scene:  7bc286c1b6


array([ 0,  2, 14, 15, 16, 44, 45, 46, 49, 64, 65], dtype=int32)

In [22]:
np.unique(dataaset[9][2][:, 2])

Scene:  7bc286c1b6


array([0], dtype=int32)

In [14]:
for i in range(280):
    data_collate = collater([dataaset[i]])
    labels = data_collate[1][0]['labels']
    invalid_labels = labels[labels>=100]
    if invalid_labels.shape[0]>0:
        print(invalid_labels)

Scene:  0a5c013435
Scene:  0a7cc12c0e
Scene:  0a76e06478
Scene:  0a184cf634
Scene:  0b031f3119
Scene:  0cf2e9402d
Scene:  0e75f3c4d9
Scene:  1a8e0d78c0
Scene:  1a130d092a
Scene:  1ae9e5d2a6
Scene:  1b9692f0c7
Scene:  1b75758486
Scene:  1c4b893630
Scene:  1d003b07bd
Scene:  1f7cbbdde1
Scene:  2a496183e1
Scene:  2b1dc6d6a5
Scene:  2e74812d00
Scene:  3c95c89d61
Scene:  3e928dc2f6
Scene:  3f1e1610de
Scene:  4a1a3a7dc5
Scene:  4ba22fa7e4
Scene:  4c5c60fa76
Scene:  4ea827f5a1
Scene:  5a269ba6fe
Scene:  5d152fab1b
Scene:  5fb5d2dbf2
Scene:  6b40d1a939
Scene:  6cc2231b9c
Scene:  6d89a7320d
Scene:  6ee2fc1070
Scene:  6f1848d1e3
Scene:  7cd2ac43b4
Scene:  7e7cd69a59
Scene:  7e09430da7
Scene:  7eac902fd5
Scene:  7f4d173c9c
Scene:  07f5b601ee
Scene:  07ff1c45bb
Scene:  8a20d62ac0
Scene:  8a35ef3cfe
Scene:  8b2c0938d6
Scene:  8b5caf3398
Scene:  08bbbdcc3d
Scene:  8be0cd3817
Scene:  8d563fc2cc
Scene:  8e00ac7f59
Scene:  8e6ff28354
Scene:  8f82c394d6
Scene:  9b74afd2d2
Scene:  09bced689e
Scene:  9f21

In [25]:
data_collate

(<datasets.utils.NoGpu at 0x7f639414db40>,
 [{'labels': tensor([67,  3, 37, 79, 27,  1, 31, 31, 44, 43]),
   'masks': tensor([[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ...,  True,  True,  True]]),
   'segment_mask': tensor([[True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True],
           [True]]),
   'point2segment': tensor([0, 0, 0,  ..., 0, 0, 0])}],
 ['data'])

In [30]:
import yaml
nodel_cfg_file = "/workspace/Mask3D_adapted/test/model_conf.yaml"
def load_hydra_config(config_path):
    with open(config_path, "r") as f:
        config = OmegaConf.load(f)
    return config
config = load_hydra_config(nodel_cfg_file)

In [31]:
mask3d_model = Mask3D(
    **config
).cuda()

In [43]:
def to_cuda(x):
    r"""Move all tensors to cuda."""
    if isinstance(x, list):
        x = [to_cuda(item) for item in x]
    elif isinstance(x, tuple):
        x = (to_cuda(item) for item in x)
    elif isinstance(x, dict):
        x = {key: to_cuda(value) for key, value in x.items()}
    elif isinstance(x, torch.Tensor):
        x = x.cuda()
    return x

In [44]:
data_batch_cuda = to_cuda(data_batch)
data, target, file_names = data_batch_cuda

In [45]:
raw_coordinates = data.features[:, -3:]
data.features = data.features[:, :-3]

In [46]:
data.features.shape

torch.Size([244314, 3])

In [47]:
data = ME.SparseTensor(
    coordinates=data.coordinates,
    features=data.features,
    device=torch.device('cuda'),
)

In [48]:
data.device

device(type='cuda', index=0)

In [49]:
output = mask3d_model.forward(              
    data,
    point2segment=[
        target[i]["point2segment"] for i in range(len(target))
    ],
    raw_coordinates=raw_coordinates,
)


In [50]:
label_offset = dataaset.label_offset
prediction = output["aux_outputs"]
prediction.append(
    {
        "pred_logits": output["pred_logits"],
        "pred_masks": output["pred_masks"],
    }
)

In [52]:
prediction[-1][
    "pred_logits"
] = torch.functional.F.softmax(
    prediction[-1]["pred_logits"], dim=-1
)[
    ..., :-1
]

In [53]:
all_pred_classes = list()
all_pred_masks = list()
all_pred_scores = list()
all_heatmaps = list()
all_query_pos = list()

In [56]:
def get_mask_and_scores(
    mask_cls, mask_pred, num_queries=100, num_classes=18, device=None
):
    labels = (
        torch.arange(num_classes, device=device)
        .unsqueeze(0)
        .repeat(num_queries, 1)
        .flatten(0, 1)
    )

    scores_per_query, topk_indices = mask_cls.flatten(0, 1).topk(
        100, sorted=True
    )
    labels_per_query = labels[topk_indices]
    topk_indices = topk_indices // num_classes
    mask_pred = mask_pred[:, topk_indices]

    result_pred_mask = (mask_pred > 0).float()
    heatmap = mask_pred.float().sigmoid()

    mask_scores_per_image = (heatmap * result_pred_mask).sum(0) / (
        result_pred_mask.sum(0) + 1e-6
    )
    score = scores_per_query * mask_scores_per_image
    classes = labels_per_query

    return score, result_pred_mask, classes, heatmap

In [58]:
mask3d_model.num_classes

101

In [59]:
offset_coords_idx = 0
classes_list = []
for bid in range(len(prediction[-1]["pred_masks"])):
    masks = (
        prediction[-1]["pred_masks"][bid]
        .detach()
        .cpu()
    )


    scores, masks, classes, heatmap = get_mask_and_scores(
        prediction[-1]["pred_logits"][bid]
        .detach()
        .cpu(),
        masks,
        prediction[-1]["pred_logits"][bid].shape[
            0
        ],
        mask3d_model.num_classes - 1,
    )

    classes_list.append(classes)

    # masks = self.get_full_res_mask(
    #     masks,
    #     inverse_maps[bid],
    #     target_full_res[bid]["point2segment"],
    # )

    # heatmap = self.get_full_res_mask(
    #     heatmap,
    #     inverse_maps[bid],
    #     target_full_res[bid]["point2segment"],
    #     is_heatmap=True,
    # )

    # if backbone_features is not None:
    #     backbone_features = self.get_full_res_mask(
    #         torch.from_numpy(backbone_features),
    #         inverse_maps[bid],
    #         target_full_res[bid]["point2segment"],
    #         is_heatmap=True,
    #     )
    #     backbone_features = backbone_features.numpy()

    # masks = masks.numpy()
    # heatmap = heatmap.numpy()

    # sort_scores = scores.sort(descending=True)
    # sort_scores_index = sort_scores.indices.cpu().numpy()
    # sort_scores_values = sort_scores.values.cpu().numpy()
    # sort_classes = classes[sort_scores_index]

    # sorted_masks = masks[:, sort_scores_index]
    # sorted_heatmap = heatmap[:, sort_scores_index]

    # keep_instances = set()
    # pairwise_overlap = sorted_masks.T @ sorted_masks
    # normalization = pairwise_overlap.max(axis=0)
    # norm_overlaps = pairwise_overlap / normalization

    # for instance_id in range(norm_overlaps.shape[0]):
    #     # filter out unlikely masks and nearly empty masks
    #     # if not(sort_scores_values[instance_id] < 0.3 or sorted_masks[:, instance_id].sum() < 500):
    #     if not (
    #         sort_scores_values[instance_id]
    #         < self.config.general.scores_threshold
    #     ):
    #         # check if mask != empty
    #         if not sorted_masks[:, instance_id].sum() == 0.0:
    #             overlap_ids = set(
    #                 np.nonzero(
    #                     norm_overlaps[instance_id, :]
    #                     > self.config.general.iou_threshold
    #                 )[0]
    #             )

    #             if len(overlap_ids) == 0:
    #                 keep_instances.add(instance_id)
    #             else:
    #                 if instance_id == min(overlap_ids):
    #                     keep_instances.add(instance_id)

    # keep_instances = sorted(list(keep_instances))
    # all_pred_classes.append(sort_classes[keep_instances])
    # all_pred_masks.append(sorted_masks[:, keep_instances])
    # all_pred_scores.append(sort_scores_values[keep_instances])
    # all_heatmaps.append(sorted_heatmap[:, keep_instances])

/tmp/ipykernel_84/2742014025.py:15: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_indices = topk_indices // num_classes


In [60]:
classes_list

[tensor([71, 51, 71, 71, 51, 51, 71, 71, 71, 71, 71, 71, 71, 71, 51, 71, 71, 71,
         71, 71, 71, 71, 51, 51, 71, 71, 51, 71, 71, 51, 51, 51, 71, 71, 71, 71,
         51, 71, 51, 71, 51, 71, 51, 51, 51, 71, 71, 71, 71, 71, 71, 51, 51, 71,
         71, 71, 71, 51, 51, 71, 71, 71, 71, 71, 71, 71, 71, 71, 51, 51, 71, 71,
         71, 71, 71, 71, 71, 71, 71, 71, 51, 51, 71, 51, 51, 51, 51, 71, 71, 71,
         51, 51, 51, 71, 71, 51, 71, 51, 51, 51]),
 tensor([71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71,
         71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71,
         71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71,
         71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71,
         71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71,
         71, 71, 71, 71, 71, 71, 71, 71, 71, 71])]